In [3]:
import json
import os

In [4]:
import spacy

In [5]:
#Load the job description data 
data = json.load(open(os.path.expanduser("~/Desktop/Thesis/Data/output_V1.1.json")))

In [6]:
#View the first entry in the "Content" column 
data[0]["CONTENT"]

'<p>Wir sind ein erfolgreiches und unabhängiges Ingenieurbüro in Schönried.<br><br>Zur Verstärkung unseres Teams und um die vielfältigen, interessanten Objekte optimal bearbeiten zu können, suchen wir</p>\n<h2>Bauingenieur (m/w)</h2>\n<h3>Ihre Arbeitsgebiete</h3>\n<p>Konstruktiver Ingenieurbau sowohl in Projektierung&nbsp; als auch in der Bauleitung allgemeiner Tief- und Strassenbau.</p>\n<h3>Ihr Profil</h3>\n<p>Sie verfügen über ein abgeschlossenes Studium als Bauingenieur (ETH/TU/FH).</p>\n<p>Sie möchten Erfahrungen in Projektierung und Bauleitung sammeln.</p>\n<p>Sie sind umgänglich und wollen sich weiterentwickeln, sind gewohnt mit Druck umzugehen und können auch in hektischen Situationen die Ruhe bewahren.</p>\n<h3>Ihre Chancen</h3>\n<p>Sie können Ihre Kenntnisse und Erfahrungen in einem kleinen motivierten Team, mit modernen Arbeitsgeräten und interessanten Objekten erweitern.<br><br>Sie erhalten die Gelegenheit, an der Aufgabe zu wachsen.</p>\n<h3>Weitere Informationen und Bewer

In [7]:
#Count how many job descriptions contain the word "the"
probably_english = [x["CONTENT"] for x in data if "the " in x["CONTENT"].lower()]
len(probably_english)

1992

In [8]:
#Define a function that takes in job descriptions and an array of common words that appends to an array every time it finds a common word
def wordMatches(phrase, commonWords):
    return sum([phrase.count(w+" ") for w in commonWords])

In [9]:
common_english = ["the", "be", "to", "of", "and", "a", "in", "that", "have", "it",]
english_matches = [wordMatches(x["CONTENT"], common_english) for x in data]

In [10]:
common_french = ["le", "de", "un", "être", "et", "à", "il", "avoir", "ne", "son"]
french_matches = [wordMatches(x["CONTENT"], common_french) for x in data]

In [11]:
common_german = ["der", "die", "das", "und", "sein", "in", "ein", "zu", "haben", "werden"]
german_matches = [wordMatches(x["CONTENT"], common_german) for x in data]

In [12]:
english = []
french = []
german = []
missed = []
for i, (e, f, g) in enumerate(zip(english_matches, french_matches, german_matches)):
    if e > f and e > g:
        english.append(data[i])
    elif f > e and f > g:
        french.append(data[i])
    elif g > e and g > f:
        german.append(data[i])
    else:
        missed.append(data[i])

In [13]:
#we can see that there are some descriptions that don't contain enough common words in any language. we will have to see if its best to delete these.
print(len(english)+len(german)+len(french), len(data))

8115 8565


In [14]:
#Looking at first job description in English
print(english[0]["CONTENT"])

Schedule: Full-time

Job ID: REQ20000224


Brief Description:

Responsible for implementation and maintenance of the Quality Assurance Program throughout the UBC and assisting with the implementation and maintenance of the UBC Quality Management System within UBC Pharma Services.

Specific Job Duties:


* Design and implement and manage an internal audit system of clinical and Pharmacovigilance processes

* Review existing SOPs/project specific plans and audit processes to ensure that staff are compliant with process

* Create/develop and present training to staff on applicable regulatory requirements

* Document audit findings/observations/recommendations in an audit report

* Track audit responses from teams

* Review audit findings/observations/recommendations and identify process improvement initiatives

* Manage and/or conduct audits of internal processes/documentation to assess compliance with applicable regulations as well as UBC SOPs and contractual requirements

* Manage and/o

In [15]:
import pandas as pd

In [ ]:
# Report the number of sentences in the first job description.

In [16]:
by_sentence = english[0]["CONTENT"].split("\n")
print("Number of sentences is", len(by_sentence))
print("The first 10 are", by_sentence[:10])

Number of sentences is 59
The first 10 are ['Schedule: Full-time', '', 'Job ID: REQ20000224', '', '', 'Brief Description:', '', 'Responsible for implementation and maintenance of the Quality Assurance Program throughout the UBC and assisting with the implementation and maintenance of the UBC Quality Management System within UBC Pharma Services.', '', 'Specific Job Duties:']


In [17]:
import logging

logging.basicConfig(level=logging.INFO)

In [18]:
#Just to try some basic manipulation/sentence similarity matching, lets look at some of the jobs related to "sustainability"
sustain = []
for e in english:
    if "Sustain" in e["JOBTITEL"]:
        sustain.append(e)

In [19]:
#Looks like one of the jobs related to sustainability is for Starbucks. We will name this "starbucks"
starbucks = sustain[0]["CONTENT"]
starbucks_sentence = [x.strip("*") for x in starbucks.split("\n") if x != '']

In [20]:
import pandas as pd


In [21]:
#Load the activities file and call it df
df = pd.read_csv(os.path.expanduser("~/Desktop/Thesis/Data/Activities.csv"), error_bad_lines=False)

In [22]:
#Rename the columns of the activities file so they are more usable. We will be using the activities as desribed by short sentences
newdf = df.rename(columns={'O*Net\nMétiers verts' : "label", 'O"Net: Détails des activités pour les métiers verts' : "activities"})

In [23]:
newdf[["label", "activities"]][~newdf["label"].isna()]

,label,activities
3,4.A.1.a.1.G01,Collect geographic information systems (GIS) d...
4,4.A.1.a.1.G02,Collect geological data.
5,4.A.1.a.1.G03,Collect data about green practices or programs.
6,4.A.1.a.1.G04,Collect energy use data.
7,4.A.1.a.1.G05,Collect product or environmental samples for t...
...,...,...
222,4.A.4.b.6.G05,Advise others on green engineering or design i...
223,4.A.4.b.6.G06,Advise others on green investments.
224,4.A.4.b.6.G07,Advise others on environmental remediation or ...
225,4.A.4.b.6.G08,Provide technical guidance on green projects.


In [24]:
#Find all the activities that contain the word "sustain". We will try to see if they share similarity with the starbucks job.
sustain_acts = [a for a in newdf["activities"] if "sustain" in str(a).lower()]

In [25]:
#Look briefly at all the activites
newdf["activities"]

0      Where and how are the information and data gai...
1       Observing, receiving, and otherwise obtaining...
2                                                    NaN
3      Collect geographic information systems (GIS) d...
4                               Collect geological data.
                             ...                        
225        Provide technical guidance on green projects.
226    Performing day-to-day administrative tasks suc...
227                                      Obtain permits.
228     Recruiting, interviewing, selecting, hiring, ...
229     Monitoring and controlling resources and over...
Name: activities, Length: 230, dtype: object

In [26]:
#Look at just the activities that contain the word "sustain"
print(sustain_acts)
print("The number of activities that contain the word sustain are",len(sustain_acts))

['Monitor sustainability indicators or other measures of green program effectiveness.', 'Develop environmental sustainability plans or projects. ', 'Design sustainable industrial processes or technologies.']
The number of activities that contain the word sustain are 3


In [27]:
#Take a look at the Starbucks description again
print(starbucks)

Starbucks Coffee Trading Company (SCTC), based in Lausanne, an affiliate company of Starbucks Coffee Company, is responsible for all of Starbucks green coffee purchasing functions and continues an unwavering commitment to sourcing, roasting and delivering only the highest quality Arabica coffees.

In order to strengthen our current organization, we are recruiting aSustainability Specialist (full-time).

Responsibilities and essential job functions include but are not limited to the following:


* Subject to coffee and coffee production experience, will help drive the evolution and development of our Ethical Sourcing Programs standards and indicators.

* Provide support and coordination in planning events related to all aspects of the administration of Ethical Coffee Sourcing Programs program. Support 3rd Party audit activity preparation, verification monitoring and shadow audits.

* Track annual purchasing data for further reporting into Global Responsibility. Transparency and traceabi

In [28]:
print(starbucks_sentence)

['Starbucks Coffee Trading Company (SCTC), based in Lausanne, an affiliate company of Starbucks Coffee Company, is responsible for all of Starbucks green coffee purchasing functions and continues an unwavering commitment to sourcing, roasting and delivering only the highest quality Arabica coffees.', 'In order to strengthen our current organization, we are recruiting aSustainability Specialist (full-time).', 'Responsibilities and essential job functions include but are not limited to the following:', ' Subject to coffee and coffee production experience, will help drive the evolution and development of our Ethical Sourcing Programs standards and indicators.', ' Provide support and coordination in planning events related to all aspects of the administration of Ethical Coffee Sourcing Programs program. Support 3rd Party audit activity preparation, verification monitoring and shadow audits.', ' Track annual purchasing data for further reporting into Global Responsibility. Transparency and 

In [29]:
from scipy import spatial
import numpy as np

In [30]:
from sent2vec.vectorizer import Vectorizer

/Users/emilysharata/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [43]:
vectorizer = Vectorizer()

In [44]:
vectorizer.vectors

[]

In [45]:
#vectorizer.bert(sustain_acts + starbucks_sentence)

In [46]:
#vectors_bert_star = vectorizer.vectors

In [47]:
#print(len(vectors_bert_star),len(starbucks_sentence))
#vectors_bert_star

27 24


array([[ 0.10083248, -0.2559276 ,  0.25373894, ..., -0.04750134,
         0.49411827, -0.01433489],
       [ 0.22708401, -0.24253352,  0.19259986, ..., -0.01968886,
         0.5213354 , -0.06281801],
       [ 0.18757936, -0.20222832,  0.21969041, ..., -0.03970376,
         0.52207696, -0.01737296],
       ...,
       [-0.3808492 , -0.17122799, -0.01378075, ..., -0.02913184,
         0.45424843,  0.16941491],
       [-0.09688131, -0.0369665 ,  0.0582107 , ..., -0.02043307,
         0.21404622,  0.638735  ],
       [ 0.17177413, -0.1992814 ,  0.2103268 , ...,  0.04191228,
         0.6096772 , -0.0271646 ]], dtype=float32)

In [51]:
vectorizer.bert(sustain_acts+starbucks_sentence)
vectors_bert_all = vectorizer.vectors

In [149]:
vectors_bert_all

array([[ 0.00998792, -0.515236  , -0.4422387 , ..., -0.28360677,
         0.97924036,  0.10759158],
       [ 0.24797446, -0.3398302 , -0.206575  , ..., -0.23113875,
         1.0178344 ,  0.1125699 ],
       [ 0.09938529, -0.19218685, -0.13090894, ..., -0.14742354,
         0.92989254,  0.28508624]], dtype=float32)

In [ ]:
from scipy import spatial

In [48]:
df = pd.DataFrame(columns=["sentence", "activity1", "activity2", "activity3"])

In [49]:
messages = sustain_acts + starbucks_sentence

In [52]:
for i, v in enumerate(vectors_bert_all):
    dist_1 = spatial.distance.cosine(vectors_bert_all[0], v)
    dist_2 = spatial.distance.cosine(vectors_bert_all[1], v)
    dist_3 = spatial.distance.cosine(vectors_bert_all[2], v)
    df = df.append({"sentence" : messages[i], "activity1" : dist_1, "activity2" : dist_2, "activity3" : dist_3}, ignore_index=True)

In [53]:
for i in range(3):
    label = "activity%i" % (i+1)
    senmatch = df[label][3:].idxmin()
    print(senmatch)
    print("%s is " % label, messages[i])
    print("--> Best match is", df["sentence"][senmatch])

11
activity1 is  Monitor sustainability indicators or other measures of green program effectiveness.
--> Best match is  Multi-roaster sustainability program development and implementation.
11
activity2 is  Develop environmental sustainability plans or projects. 
--> Best match is  Multi-roaster sustainability program development and implementation.
11
activity3 is  Design sustainable industrial processes or technologies.
--> Best match is  Multi-roaster sustainability program development and implementation.


In [179]:
maxvals = pd.DataFrame(columns=["sentence1", "sentence2", "sentence3"])
maxvals

,sentence1,sentence2,sentence3


In [180]:

for i, v in enumerate(vectors_bert_all[3:]):
    dist_1 = spatial.distance.cosine(vectors_bert_all[0], v)
    dist_2 = spatial.distance.cosine(vectors_bert_all[1], v)
    dist_3 = spatial.distance.cosine(vectors_bert_all[2], v)
    maxvals = maxvals.append({"sentence1" : dist_1, "sentence2" : dist_2, "sentence3" : dist_3}, ignore_index=True)

In [186]:
print(maxvals["sentence1"].idxmax())
print(maxvals["sentence2"].idxmax())
print(maxvals["sentence3"].idxmax())

print(sustain_acts[1], starbucks_sentence[45])
print(sustain_acts[0], starbucks_sentence[0])

0
45
45
Develop environmental sustainability plans or projects.  * Ability to lead multiple projects and working groups. Communicate effectively to a wide range of internal and external stakeholders and be able to identify inefficiencies and clarify misalignments in a analytical manner that generates fair and prompt resolution of problems.
Monitor sustainability indicators or other measures of green program effectiveness. Starbucks Coffee Trading Company (SCTC), based in Lausanne, an affiliate company of Starbucks Coffee Company, is responsible for all of Starbucks green coffee purchasing functions and continues an unwavering commitment to sourcing, roasting and delivering only the highest quality Arabica coffees.


In [191]:
print("Average sentence1", np.sum(maxvals["sentence1"])/len(maxvals))
print("Max sentence1", np.max(maxvals["sentence1"]))
print(np.sum(maxvals["sentence1"])/len(maxvals))
print(np.sum(maxvals["sentence2"])/len(maxvals))

Average sentence1 0.08423543969790141
Max sentence1 0.2450575828552246
0.08423543969790141
0.079433955528118
